# DAMLAS - Machine Learning At Scale
## Assignment - HW2
Data Analytics and Machine Learning at Scale
Target, Bangalore

---
__Name:__  *Your Name Goes Here*   
__Class:__ DAMLAS (Section *Your Section Goes Here*, e.g., Fall 2016)     
__Email:__  *Your Target Email User Goes Here*@Target.com     
__Week:__   01

# Table of Contents <a name="TOC"></a> 

1.  [HW Introduction](#1)   
2.  [HW References](#2)
3.  [HW  Problems](#3)   
    1.0.  [HW2.1](#1.1)   
    1.2.  [HW2.2](#1.2)   
    1.3.  [HW2.3](#1.3)    
    1.4.  [HW2.4](#1.4)    
    


<a name="1">
# 1 Instructions
[Back to Table of Contents](#TOC)
* Homework submissions are due by Wednesday, 08/30/2016 at 10:30AM.
* Word count in Hadoop Streaming with Hadoop is described (should work on your Virtual Machines)


* Prepare a single Jupyter notebook (not a requirment), please include questions, and question numbers in the questions and in the responses.
Submit your homework notebook via the following form:

   + [Submission Link - Google Form](http://goo.gl/forms/er3OFr5eCMWDngB72)


### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.

<a name="2">
# 2 Useful References
[Back to Table of Contents](#TOC)

* [Hadoop Primer](https://www.dropbox.com/s/g84vzatuyalhe9y/Lecture%203-Hadoop-Intro-2016-06-13.pdf?dl=0)
* [Word Count example using Hadoop Streaming and  Python](http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/
)
* [Everything you want to know about dictionaries in Python](http://openbookproject.net/thinkcs/python/english3e/dictionaries.html)
---


## Hadoop Streaming
Hadoop streaming is a utility that comes with the Hadoop distribution. The utility allows you to create and run Map/Reduce jobs with any executable or script as the mapper and/or the reducer. For example:

## How Streaming Works

In the above example, both the mapper and the reducer are executables that read the input from stdin (line by line) and emit the output to stdout. The utility will create a Map/Reduce job, submit the job to an appropriate cluster, and monitor the progress of the job until it completes.

When an executable is specified for mappers, each mapper task will launch the executable as a separate process when the mapper is initialized. As the mapper task runs, it converts its inputs into lines and feed the lines to the stdin of the process. In the meantime, the mapper collects the line oriented outputs from the stdout of the process and converts each line into a key/value pair, which is collected as the output of the mapper. By default, the prefix of a line up to the first tab character is the key and the rest of the line (excluding the tab character) will be the value. If there is no tab character in the line, then entire line is considered as key and the value is null. However, this can be customized, as discussed later.

When an executable is specified for reducers, each reducer task will launch the executable as a separate process then the reducer is initialized. As the reducer task runs, it converts its input key/values pairs into lines and feeds the lines to the stdin of the process. In the meantime, the reducer collects the line oriented outputs from the stdout of the process, converts each line into a key/value pair, which is collected as the output of the reducer. By default, the prefix of a line up to the first tab character is the key and the rest of the line (excluding the tab character) is the value. However, this can be customized, as discussed later.

This is the basis for the communication protocol between the Map/Reduce framework and the streaming mapper/reducer.

## References for Hadoop Streaming
* [Hadoop Streaming manual](https://hadoop.apache.org/docs/r1.2.1/streaming.html#Hadoop+Streaming)

## Anatomy of a Hadoop Job

For more details see:

http://ercoppa.github.io/HadoopInternals/AnatomyMapReduceJob.html

In [ ]:
pip install mrjob

In [ ]:
export PATH="/home/username/anaconda/bin:$PATH"

## locate the hadoop streaming jar

In [1]:
! find / -name "hadoop*streaming*" -print 1> tmp.hadoopJar 2> /dev/null

^C


In [1]:
mydict = {"a":1, "b":2, "d":4, "c":3}          
sorted([(value,key) for (key,value) in mydict.items()])

#[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [2]:
!cat tmp.hadoopJar

/Users/jshanahan/Google Drive/JShanahan/Publications/Conferences/STRATA-2012/tutorial_large_scale_mining/hadoop-0.20.2/contrib/streaming/hadoop-0.20.2-streaming.jar
/Users/jshanahan/Publications/Conferences/STRATA-2012/tutorial_large_scale_mining/hadoop-0.20.2/contrib/streaming/hadoop-0.20.2-streaming.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/doc/hadoop/hadoop-streaming
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/sources/hadoop-streaming-2.6.0-sources.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/sources/hadoop-streaming-2.6.0-test-sources.jar


In [15]:
!ls -l /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar



-rw-r--r-- 1 root root 107707 Jun 16 13:00 /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar


In [5]:
!mkdir WordCount

In [5]:
%%writefile WordCount/mapper.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    for word in line.split():
        print '%s\t%s' % (word, 1)


Writing WordCount/mapper.py


In [6]:
%%writefile WordCount/reducer.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)


Writing WordCount/reducer.py


In [8]:
%load WordCount/reducer.py  #Load code into the current frontend

ValueError: 'WordCount/reducer.py #Load code into the current frontend' was not found in history, as a file, url, nor in the user namespace.

In [9]:
!chmod a+x WordCount/mapper.py

In [10]:
!chmod a+x WordCount/reducer.py

In [7]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py 

/bin/sh: WordCount/mapper.py: Permission denied


In [11]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py | sort -k1,1 | WordCount/reducer.py| sort -k2,2nr 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are you
reporter:counter:Reducer Counters,Calls,1
foo	3
quux	2
bar	1
labs	1


In [12]:
!hdfs dfs

Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] <localsrc> ... <dst>]
	[-copyToLocal [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] <path> ...]
	[-cp [-f] [-p | -p[topax]] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] <src> <localdst>]
	[-help [cmd ...]]
	[-ls [-d] [-h] [-R] [<path> ...]]
	[-mkdir [-p] <path> ...]
	[-moveFromLocal <localsrc> ... <dst>]
	[-moveToLocal <src> <localdst>]
	[-mv <src> ... <dst>]
	[-put [-f] [-p] 

In [14]:
!hdfs dfs -ls

In [23]:
%%writefile testWordCountInput.txt
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello

Overwriting testWordCountInput.txt


In [22]:
!cd WordCount/

In [25]:
!hdfs dfs -rm testWordCountInput.txt 
!hdfs dfs -copyFromLocal testWordCountInput.txt 
!hdfs dfs -rm -r wordcount-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks/WordCount"

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -mapper /home/cloudera/WordCount/mapper.py \
   -reducer /home/cloudera/WordCount/reducer.py \
   -combiner /home/cloudera/WordCount/reducer.py \
   -input testWordCountInput.txt \
   -output wordcount-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2  depecated
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

Deleted testWordCountInput.txt
Deleted wordcount-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob7511691146345687632.jar tmpDir=null
16/08/26 20:46:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 20:46:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 20:47:00 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/08/26 20:47:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/26 20:47:00 WARN hdfs.DFSClient: Caught exception 
java.lang.In

In [26]:
#have a look at the input
!echo  "\n---------------------------\n"
!hdfs dfs -cat testWordCountInput.txt
!echo  "\n---------------------------\n"
# Wordcount output
!hdfs dfs -cat wordcount-output/part-0000*

\n---------------------------\n
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello\n---------------------------\n
Hello	1
jimi	1
this	1
three	1
Jimi	3
hello	2
is	1
who	1


# 3.  HW1  <a name="3"></a>
[Back to Table of Contents](#TOC)

* Dont forget to create a new subdirectory for this homework (on your local machine/virtual machine, and on HDFS)
---
## 3.  HW2.1  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

Change the mapper.py/reducer.py combination from the the above WordCount example so that you get the longest word present in the text version of Alice’s Adventures in Wonderland. (You can obtain a free plain text version of the book, along with many others, from [here](http://www.gutenberg.org.). First use one reducer and report your result. HINT: from emit records of the form: "longestWord\theLongWordEver\t15".  What happens if you use 3 reducers? Anything change with respect to your solution.

In [ ]:
!curl 'http://www.gutenberg.org/cache/epub/11/pg11.txt' -o alicesTExtFilename.txt

## 3.  HW2.2  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

Change the mapper.py/reducer.py combination so that you get only the number of words starting with an uppercase letter, and the number of words starting with a lowercase letter. In other words, you need an output file with only 2 lines, one giving you the number of words staring with a lowercase ('a' to 'z'), and the other line indicating the number of words starting with an uppercase letter ('A' to 'Z'). 

## 3.  HW2.3  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

The data file at http://www.metoffice.gov.uk/climate/uk/stationdata/heathrowdata.txt (mirrored here) is part of a set of text files cataloguing temperatures collected in different cities of the UK. The one for the URL given is for Heathrow (London Airport). Typically each city is held in one data file. You can access the whole set from [here](http://www.metoffice.gov.uk/climate/uk/stationdata/).
Write a MapReduce application with a mapper.py and reduce.py that uses only the text file for Heathrow as input and that outputs the lowest temperature ever recorded and the year this event happened. 


## 3.  HW2.4  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

Reproduce the following webpage (Word Count in Hadoop Streaming over different examples) in a Jupyter notebook:

* http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/


# END OF Homework

In [ ]:
# This call to Hadoop does NOT work as anticpated
# IdentityMapper, IdentityReducer do not trigger the Hadoop framework to sort properly

!hdfs dfs -rm -r wordcount-output-sorted
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
   -input wordcount-output \
   -output wordcount-output-sorted  \
   -numReduceTasks 1 \
   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
   -reducer org.apache.hadoop.mapred.lib.IdentityReducer \

#DOES not work in streaming mode
#   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
#  -reducer org.apache.hadoop.mapred.lib.IdentityReducer \


In [29]:
#Custom partitioner works 
# partition based on first parts
#sort numerically decreasing on the third part
!hdfs dfs -rm -r wordcount-output-sorted 
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
  -D mapreduce.job.reduces=3 \
  -input wordcount-output \
  -output wordcount-output-sorted \
    -mapper /bin/cat \
   -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner
    
#   -mapper identityMapper.py \

#Can also use -mapper /bin/cat \
#-mapper /bin/cat \
#-reducer /bin/cat \


Deleted wordcount-output-sorted
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob9044814104207183172.jar tmpDir=null
16/08/26 21:29:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 21:29:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 21:30:01 INFO mapred.FileInputFormat: Total input paths to process : 3
16/08/26 21:30:01 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/08/26 21:30:01 INFO mapreduce.JobSubmitter: number of splits:3
16/08/26 21:30:02 INFO mapr

In [30]:
!hdfs dfs -ls wordcount-output-sorted/part* 
!hdfs dfs -cat wordcount-output-sorted/part* 


-rw-r--r--   1 cloudera cloudera          0 2016-08-26 21:31 wordcount-output-sorted/part-00000
-rw-r--r--   1 cloudera cloudera         42 2016-08-26 21:31 wordcount-output-sorted/part-00001
-rw-r--r--   1 cloudera cloudera         22 2016-08-26 21:31 wordcount-output-sorted/part-00002
Jimi	3	
Hello	1	
jimi	1	
this	1	
three	1	
hello	2	
is	1	
who	1	
